# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4922, uptime 0:00:37
mini_beamline                    RUNNING   pid 4923, uptime 0:00:37
random_walk                      RUNNING   pid 4924, uptime 0:00:37
random_walk_horiz                RUNNING   pid 4925, uptime 0:00:37
random_walk_vert                 RUNNING   pid 4926, uptime 0:00:37
simple                           RUNNING   pid 4927, uptime 0:00:37
thermo_sim                       RUNNING   pid 4928, uptime 0:00:37
trigger_with_pc                  FATAL     Exited too quickly (process log may have details)


In [2]:
%run scripts/beamline_configuration.py

/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/traitlets/traitlets.py:3036: FutureWarning: --rc={'figure.dpi': 96} for dict-traits is deprecated in traitlets 5.0. You can pass --rc <key=value> ... multiple times to add items to a dict.
  FutureWarning,


/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/pims/image_reader.py:26: RuntimeWarning: PIMS image_reader.py could not find scikit-image. Falling back to matplotlib's imread(), which uses floats instead of integers. This may break your scripts. 
(To ignore this warning, include the line "warnings.simplefilter("ignore", RuntimeWarning)" in your script.)
  warnings.warn(RuntimeWarning(ski_preferred))


## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2021-02-05 23:34:27
Persistent Unique Scan ID: 'e4305d45-cee5-4a8c-afdb-c725c2dd3be2'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 23:34:27.5 |     -1.000 |      98729 |
|         2 | 23:34:27.6 |     -0.500 |      99056 |
|         3 | 23:34:27.6 |      0.000 |      99013 |
|         4 | 23:34:27.6 |      0.500 |      98360 |


|         5 | 23:34:27.6 |      1.000 |      96809 |


+-----------+------------+------------+------------+
generator scan ['e4305d45'] (scan num: 3)





('e4305d45-cee5-4a8c-afdb-c725c2dd3be2',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/e4305d45-cee5-4a8c-afdb-c725c2dd3be2-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2021-02-05 23:34:27
Persistent Unique Scan ID: '9fca197d-daba-4dc7-8f6f-929047fea0dd'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 23:34:28.1 |          -1 | 1067982668 |


|         2 | 23:34:28.2 |          -0 | 1059640805 |


|         3 | 23:34:28.3 |           0 | 1056182675 |
|         4 | 23:34:28.4 |           0 | 1054281506 |


|         5 | 23:34:28.5 |           1 | 1053296968 |
+-----------+------------+-------------+------------+
generator scan ['9fca197d'] (scan num: 4)





('9fca197d-daba-4dc7-8f6f-929047fea0dd',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2021-02-05 23:34:28
Persistent Unique Scan ID: 'f3178894-20d2-4f6f-8ce9-6489819bd9f2'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 23:34:29.0 | 1059762236 |


|         2 | 23:34:29.9 | 1114449628 |


|         3 | 23:34:30.9 | 1124946015 |


...

|        28 | 23:34:55.9 | 1068618700 |


|        29 | 23:34:56.9 | 1061567698 |


|        30 | 23:34:58.0 | 1116891237 |


+-----------+------------+------------+
generator count ['f3178894'] (scan num: 5)





('f3178894-20d2-4f6f-8ce9-6489819bd9f2',)